## **Filtro para selecionar as imagens do Brasil**

In [3]:
import os
import shutil
import rasterio
from rasterio.warp import transform_bounds
from shapely.geometry import box
import geopandas as gpd

# Carregar o shapefile dos limites do Brasil
def load_brazil_boundary():
    brazil = gpd.read_file('BR_Pais_2022/BR_Pais_2022.shp')
    brazil = brazil.to_crs(epsg=4326)  # Garantir que esteja no CRS EPSG:4326
    return brazil

# Verificar se uma imagem está dentro dos limites do Brasil
def is_image_within_brazil(image_path, brazil_boundary):
    with rasterio.open(image_path) as src:
        img_crs = src.crs
        bounds = src.bounds
        
        if img_crs.to_string() != 'EPSG:4326':
            bounds = transform_bounds(img_crs, 'EPSG:4326', *bounds)

        img_bbox = box(*bounds)
        return img_bbox.intersects(brazil_boundary)

# Função para copiar imagens que estão dentro do Brasil
def copy_images_in_brazil(src_dir, dest_dir, brazil_boundary):
    for root, dirs, files in os.walk(src_dir):
        for file in files:
            if file.endswith('.tif'):
                image_path = os.path.join(root, file)
                
                if is_image_within_brazil(image_path, brazil_boundary):
                    dest_path = os.path.join(dest_dir, file)
                    shutil.copy(image_path, dest_path)
                    print(f"Imagem {file} copiada para {dest_path}.")
                else:
                    print(f"Imagem {file} não está no Brasil.")

def main():
    src_dir = r"D:\CloudSen12Plus_\p2000\high\train"  # Diretório de origem
    dest_dir = r"D:\CloudSen12Plus_\p2000\high\train_br"  # Diretório de destino
    
    # Verificar se o diretório de destino existe, caso contrário, criar
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    
    brazil_boundary = load_brazil_boundary().geometry.union_all()
    copy_images_in_brazil(src_dir, dest_dir, brazil_boundary)

if __name__ == '__main__':
    main()


Imagem 20170504T083011_20170504T083011_T36SWH__ROI_2279.tif não está no Brasil.
Imagem 20170529T105621_20170529T110523_T31TCN__ROI_2864.tif não está no Brasil.
Imagem 20170530T103021_20170530T103024_T34WDS__ROI_0664.tif não está no Brasil.
Imagem 20170530T103021_20170530T103024_T34WDS__ROI_0666.tif não está no Brasil.
Imagem 20170607T095031_20170607T095921_T33SWA__ROI_0646.tif não está no Brasil.
Imagem 20170811T104021_20170811T104024_T34WEA__ROI_2887.tif não está no Brasil.
Imagem 20170829T081601_20170829T082551_T37TCE__ROI_4301.tif não está no Brasil.
Imagem 20171014T102021_20171014T102022_T35WMU__ROI_1233.tif não está no Brasil.
Imagem 20171014T102021_20171014T102022_T35WMU__ROI_4583.tif não está no Brasil.
Imagem 20171014T102021_20171014T102022_T35WMU__ROI_4584.tif não está no Brasil.
Imagem 20171022T094041_20171022T094036_T35WPN__ROI_2396.tif não está no Brasil.
Imagem 20171202T105421_20171202T105415_T31TDK__ROI_0927.tif não está no Brasil.
Imagem 20171210T083331_20171210T083717_T

## **Filtro para adicionar padding nas imagens**

In [2]:
import os
import rasterio
import numpy as np

# Função para redimensionar a imagem para as dimensões-alvo
def pad_image(image, target_size):
    height, width = image.shape[1], image.shape[2]  # Considerando formato (bandas, altura, largura)
    
    if height == target_size and width == target_size:
        return image
    
    pad_height = target_size - height
    pad_width = target_size - width
    
    pad_top = 0
    pad_bottom = pad_height
    pad_left = 0
    pad_right = pad_width
    
    padded_image = np.pad(image, ((0, 0), (pad_top, pad_bottom), (pad_left, pad_right)), mode='constant', constant_values=0)
    
    return padded_image

def adjust_transform(transform, original_width, original_height, padded_width, padded_height):
    adjusted_transform = rasterio.Affine(transform.a, transform.b, transform.c, 
                                         transform.d, transform.e, 
                                         transform.f - (padded_height - original_height) * transform.e)
    return adjusted_transform

# Função para processar imagens de um diretório, incluindo subdiretórios
def process_directory(directory, target_size_small=512, target_size_large=2048):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.tif'):
                file_path = os.path.join(root, file)
                with rasterio.open(file_path) as src:
                    image = src.read()
                    
                    original_height, original_width = image.shape[1], image.shape[2]
                    
                    if original_height == 509 and original_width == 509:
                        target_size = target_size_small
                    elif original_height == 2000 and original_width == 2000:
                        target_size = target_size_large
                    else:
                        continue

                    padded_image = pad_image(image, target_size)
                    
                    new_transform = adjust_transform(src.transform, original_width, original_height, target_size, target_size)
                    
                    profile = src.profile
                    profile.update({
                        'height': target_size,
                        'width': target_size,
                        'transform': new_transform
                    })
                    
                    # Salvar a nova imagem com um sufixo '_resized' para evitar sobrescrever
                    new_file_path = file_path.replace('.tif', '_resized.tif')
                    with rasterio.open(new_file_path, 'w', **profile) as dst:
                        dst.write(padded_image)
                        print(f"Imagem {new_file_path} redimensionada para {target_size}x{target_size}.")

# Caminho base onde o processo irá começar
base_dir = r'D:\CloudSen12_Br'

# Chama a função para processar o diretório base e todos os subdiretórios
process_directory(base_dir)

print("Redimensionamento concluído!")


Imagem D:\CloudSen12_Br\fixed\high_br\high__test__ROI_1400__20191108T131251_20191108T131245_T23LRL_resized.tif redimensionada para 512x512.
Imagem D:\CloudSen12_Br\fixed\high_br\high__train__ROI_0163__20191101T132241_20191101T132236_T22KGC_resized.tif redimensionada para 512x512.
Imagem D:\CloudSen12_Br\fixed\high_br\high__train__ROI_0610__20190405T132231_20190405T132234_T22KGU_resized.tif redimensionada para 512x512.
Imagem D:\CloudSen12_Br\fixed\high_br\high__train__ROI_0780__20190103T133219_20190103T133222_T22KDC_resized.tif redimensionada para 512x512.
Imagem D:\CloudSen12_Br\fixed\high_br\high__train__ROI_0821__20200512T133231_20200512T133229_T22KDV_resized.tif redimensionada para 512x512.
Imagem D:\CloudSen12_Br\fixed\high_br\high__train__ROI_0956__20200608T141049_20200608T141048_T21MWT_resized.tif redimensionada para 512x512.
Imagem D:\CloudSen12_Br\fixed\high_br\high__train__ROI_0956__20200703T141051_20200703T141051_T21MWT_resized.tif redimensionada para 512x512.
Imagem D:\Clou